# 1.기본 라이브러리 import

In [1]:
import time
start= time.time() #컴파일 소요 시간을 확인하기 위함.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM,Dropout
from keras.layers import RepeatVector, TimeDistributed,Flatten, Activation,ThresholdedReLU, Embedding
from keras.layers.convolutional import Conv1D,MaxPooling1D
from keras import backend as K
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import EarlyStopping

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


# 2.데이터 불러오기 및 전처리

In [3]:
train = pd.read_csv('data/sales_train_v2.csv', parse_dates=['date'], infer_datetime_format=True, dayfirst=True)
shops = pd.read_csv('data/shops.csv')
items = pd.read_csv('data/items.csv')
item_categories = pd.read_csv('data/item_categories.csv') 
test = pd.read_csv('data/test.csv') 
#sample_submission=pd.read_csv('sample_submission.csv')

In [4]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


In [5]:
max(train['date']),min(train['date'])

(Timestamp('2015-10-31 00:00:00'), Timestamp('2013-01-01 00:00:00'))

In [6]:
len(train),train.isnull().sum()

(2935849, date              0
 date_block_num    0
 shop_id           0
 item_id           0
 item_price        0
 item_cnt_day      0
 dtype: int64)

In [7]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [8]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [9]:
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [10]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [11]:
import os
print(os.listdir("./"))

['.ipynb_checkpoints', 'data', 'predict_product.ipynb', 'predict_product_v2.ipynb', 'sales_train.csv.gz', 'sample_submission.csv.gz', 'submission.csv', 'test.csv.gz']


# date 부분은 시계열 데이터로, 년도와 월로 변경 후 시계열 데이터로 변경한다.

In [12]:
df_cnt= train.groupby([train.date.apply(lambda x: x.strftime('%Y-%m')),'item_id','shop_id']).sum().reset_index()
df_cnt = df_cnt[['date','item_id','shop_id','item_cnt_day']]
df_cnt['item_cnt_day'].clip(0.,20.,inplace=True)

In [13]:
#피벗 테이블로 변경
df_cnt=df_cnt.pivot_table(index=['item_id','shop_id'],columns='date',
                            values='item_cnt_day',fill_value=0).reset_index()

In [14]:
df_cnt.head()

date,item_id,shop_id,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,...,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10
0,0,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# test 데이터 역시 생성한 피벗 테이블과 merge한다.

In [15]:
test_df_cnt = pd.merge(test,df_cnt,on=['item_id','shop_id'], how='left')

In [16]:
test_df_cnt.isnull().sum()

ID              0
shop_id         0
item_id         0
2013-01    102796
2013-02    102796
2013-03    102796
2013-04    102796
2013-05    102796
2013-06    102796
2013-07    102796
2013-08    102796
2013-09    102796
2013-10    102796
2013-11    102796
2013-12    102796
2014-01    102796
2014-02    102796
2014-03    102796
2014-04    102796
2014-05    102796
2014-06    102796
2014-07    102796
2014-08    102796
2014-09    102796
2014-10    102796
2014-11    102796
2014-12    102796
2015-01    102796
2015-02    102796
2015-03    102796
2015-04    102796
2015-05    102796
2015-06    102796
2015-07    102796
2015-08    102796
2015-09    102796
2015-10    102796
dtype: int64

In [17]:
test_df_cnt=test_df_cnt.fillna(0)

In [18]:
test_df_cnt.head()

,ID,shop_id,item_id,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,...,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10
0,0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,1,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,5,5268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
#id, item_id, shop_id 칼럼 제거
test_df_cnt = test_df_cnt.drop(labels=['ID','item_id','shop_id'],axis=1)

# 값의 범위가 큰 item_price는 MinMaxScaler로 값 변경

In [20]:
scaler = MinMaxScaler(feature_range=(0, 1))
train["item_price"] = scaler.fit_transform(train["item_price"].values.reshape(-1,1))

In [21]:
# item_price 값으로 피벗 테이블 생성

In [22]:
df_date = train.groupby([train.date.apply(lambda x: x.strftime('%Y-%m')),'item_id','shop_id']).mean().reset_index()
df_date = df_date[['date','item_id','shop_id','item_price']].pivot_table(index=['item_id','shop_id'], columns='date',values='item_price',fill_value=0).reset_index()

In [23]:
#테스트 데이터와 item_price 값을 가지는 df_date를 merge

In [24]:
df_date.head()

date,item_id,shop_id,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,...,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10
0,0,54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,55,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
sales = pd.merge(test,df_date,on=['item_id','shop_id'], how='left').fillna(0)
sales = sales.drop(labels=['ID','item_id','shop_id'],axis=1)

# 3. 학습 데이터 생성

In [26]:
y_train = test_df_cnt['2015-10'] #data-> 2013.01.01 ~ 2015.10.31 중 마지막 2015.10 predict로 학습
x_sales = test_df_cnt.drop(labels=['2015-10'],axis=1) #y train 부분 제거
x_sales = x_sales.values.reshape((x_sales.shape[0], x_sales.shape[1], 1)) #행렬 구조 변경
x_prices = sales.drop(labels=['2015-10'],axis=1) #y_train 부 제거
x_prices= x_prices.values.reshape((x_prices.shape[0], x_prices.shape[1], 1))


In [27]:
X = np.append(x_sales,x_prices,axis=2) #학습할 X 
y = y_train.values.reshape((214200, 1)) #학습할 y

In [28]:
print("Training Predictor Shape: ",X.shape)
print("Training Predictee Shape: ",y.shape)
#del y_train, x_sales; gc.collect()

Training Predictor Shape:  (214200, 33, 2)
Training Predictee Shape:  (214200, 1)


In [29]:
test_df_cnt = test_df_cnt.drop(labels=['2013-01'],axis=1)
x_test_sales = test_df_cnt.values.reshape((test_df_cnt.shape[0], test_df_cnt.shape[1], 1))
x_test_prices = sales.drop(labels=['2013-01'],axis=1)
x_test_prices = x_test_prices.values.reshape((x_test_prices.shape[0], x_test_prices.shape[1], 1))

In [30]:
test_df_cnt = np.append(x_test_sales,x_test_prices,axis=2)
#del x_test_sales,x_test_prices, price; gc.collect()
print("Test Predictor Shape: ",test.shape)

Test Predictor Shape:  (214200, 3)


# 4. 모델링

In [31]:
model_lstm = Sequential()
model_lstm.add(LSTM(16, input_shape=(X.shape[1], X.shape[2]),return_sequences=True))
model_lstm.add(Dropout(0.5))
model_lstm.add(LSTM(32))
model_lstm.add(Dropout(0.5))
model_lstm.add(Dense(1))
model_lstm.compile(optimizer="adam", loss='mse', metrics=["mse"])
print(model_lstm.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 33, 16)            1216      
_________________________________________________________________
dropout_1 (Dropout)          (None, 33, 16)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                6272      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 7,521
Trainable params: 7,521
Non-trainable params: 0
___________________

In [32]:
# batch_size=128, epochs=10

In [33]:
LSTM_PARAM = {"batch_size":128,"verbose":2,"epochs":10}

In [34]:
modelstart = time.time()

#5. 학습

In [35]:
#Validation data로 학습
if True:
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.10, random_state=1, shuffle=False)
    print("X Train Shape: ",X_train.shape)
    print("X Valid Shape: ",X_valid.shape)
    print("y Train Shape: ",y_train.shape)
    print("y Valid Shape: ",y_valid.shape)

    callbacks_list=[EarlyStopping(monitor="val_loss",min_delta=.001, patience=3,mode='auto')]
    #keras에서 자동으로 학습이 불필요할 시 종료해주는 명령어
    
    hist = model_lstm.fit(X_train, y_train,validation_data=(X_valid, y_valid),
                          callbacks=callbacks_list,**LSTM_PARAM)
    pred = model_lstm.predict(test_df_cnt)
    best = np.argmin(hist.history["val_loss"])
    print("Optimal Epoch: {}",best)
    print("Train Score: {}, Validation Score: {}".format(hist.history["loss"][best],hist.history["val_loss"][best]))

X Train Shape:  (192780, 33, 2)
X Valid Shape:  (21420, 33, 2)
y Train Shape:  (192780, 1)
y Valid Shape:  (21420, 1)
Instructions for updating:
Use tf.cast instead.
Train on 192780 samples, validate on 21420 samples
Epoch 1/10
 - 82s - loss: 0.9511 - mean_squared_error: 0.9511 - val_loss: 0.4378 - val_mean_squared_error: 0.4378
Epoch 2/10
 - 79s - loss: 0.8713 - mean_squared_error: 0.8713 - val_loss: 0.4380 - val_mean_squared_error: 0.4380
Epoch 3/10
 - 78s - loss: 0.8673 - mean_squared_error: 0.8673 - val_loss: 0.4413 - val_mean_squared_error: 0.4413
Epoch 4/10
 - 80s - loss: 0.8591 - mean_squared_error: 0.8591 - val_loss: 0.4406 - val_mean_squared_error: 0.4406
Optimal Epoch: {} 0
Train Score: 0.9511401894382823, Validation Score: 0.43778780790047594


# 6.제출양식 구성

In [36]:
submission = pd.DataFrame(pred,columns=['item_cnt_month'])
submission.to_csv('submission.csv',index_label='ID')

In [37]:
print("\nModel Runtime: %0.2f Minutes"%((time.time() - modelstart)/60))
print("Notebook Runtime: %0.2f Minutes"%((time.time() - start)/60))


Model Runtime: 6.30 Minutes
Notebook Runtime: 9.32 Minutes
